In [15]:
import Pkg 
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Plots")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
   Installed JpegTurbo_jll ──────────────── v2.1.91+0
   Installed libfdk_aac_jll ─────────────── v2.0.2+0
   Installed x265_jll ───────────────────── v3.5.0+0
   Installed GR_jll ─────────────────────── v0.72.10+0
   Installed Opus_jll ───────────────────── v1.3.2+0
   Installed Libmount_jll ───────────────── v2.35.0+0
   Installed LERC_jll ───────────────────── v3.0.0+1
   Installed LoggingExtras ──────────────── v1.0.3
   Installed Xorg_xkbcomp_jll ───────────── v1.4.6+0
   Installed Measures ───────────────────── v0.3.2
   Installed ConcurrentUtilities ────────── v2.2.1
   Installed RelocatableFolders ─────────── v1.0.1
   Installed Grisu ────────────────

In [22]:
# Process exported data from DuneAnalytics 
using CSV, DataFrames

# Corresponds to query https://dune.com/queries/3087337
# Profuces DataFrame with columns ["volume", "fee", "mean_fee"]
function dex_fee_and_volume(path)
    token_pairs = []
    for file in readdir(path)
        filename = split(file, ".")[1]
        filepath = joinpath(path, file)
        df = CSV.read(filepath, DataFrame, types=[String, Float64, Float64], silencewarnings=true)

        df = df[.!ismissing.(df.fee), :]
        df.fee = abs.(df.fee)
        df.mean_fee = df.fee ./ df.dollar_value_sold
        df.volume = copy(df.dollar_value_sold)
        select!(df, Not(:dollar_value_sold))

        df = sort(df, :volume, rev=true)
        push!(token_pairs, (filename, df))
        
    end

    return Dict(token_pairs)
end 

dex_fee_and_volume_df = dex_fee_and_volume("../queries/dex_fee_and_volume")

using Plots

for token_pair in keys(dex_fee_and_volume_df)
    token_pair_df = dex_fee_and_volume_df[token_pair][1:6,:]
    plot = bar(token_pair_df.market, token_pair_df.volume,  yaxis="Volume", xaxis="Market", size=(800, 400), legend=false)
    plot!(twinx(), token_pair_df.market, token_pair_df.mean_fee,  yaxis="Mean Fee",  size=(800, 400), color=:red, legend=false)
    savefig(joinpath("..", "output", "dex_fee_and_volume", "plot_" * token_pair * ".png"))
end


In [20]:
using CSV, DataFrames
# Corresponds to queries under the category https://dune.com/queries/3106546
# Produces DataFrame with columns ["token_pair", "total_volume"]
function dex_market_shares(path)
    ranking = []
    for file in readdir(path)
        filename = split(file, ".")[1]
        filepath = joinpath(path, file)
        df = CSV.read(filepath, DataFrame, types=[String, Float64], silencewarnings=true)

        df = sort(df, Not(Symbol(filename)), rev=true) # Sort by the numerical column
        push!(ranking, (filename, df))
        
    end
    return Dict(ranking)
end 

dex_market_shares_df = Dict(
    "fees"=> dex_market_shares("../queries/market_share/fees"), 
    "volume"=> dex_market_shares("../queries/market_share/volume")
)

using Plots


for dependent_var in keys(dex_market_shares_df)
    for category in keys(dex_market_shares_df[dependent_var])
        category_df = dex_market_shares_df[dependent_var][category]
        # remove negative entries
        category_df = category_df[category_df[!,2] .> 0, :]
        plot = pie(category_df[1:10,1], category_df[1:10,2], title=category, legend=true)
        savefig(joinpath("..", "output", "market_share", dependent_var, "plot_" * category * ".png"))
    end
end